# Computed MAF qc

The goal of the analysis is to see how many lead variants from credible sets had correctly calculated MAF.

<div class="alert alert-block alert-info"> 
<b> Reasons of MAF computation failure </b>
    <ul>
        <li><I>majorPopulationMAF == 0.0</I>  - Searched variant was found in gnomAD, but it was not reported for the searched study major population</li>
        <li><I>majorPopulationMAF is Null</I> -  Searched variant was not found in gnomAD, so AF was not reported for any population</li>
    </ul>

 </div>


## Data Loading

The data required for the analysis is loaded from the `computed maf` dataset.


In [ ]:
from gentropy.common.session import Session

from manuscript_methods.datasets import LeadVariantEffect


In [ ]:
session = Session(extended_spark_conf={"spark.driver.memory": "40G"})
lead_maf_dataset_path = "../../data/lead-maf-vep"


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/18 22:34:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/18 22:34:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/18 22:34:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
session.spark


In [5]:
dataset_maf = LeadVariantEffect.from_parquet(session, lead_maf_dataset_path)


## MAF QC

Ensure that both datasets contain the same amount of rows


In [8]:
from manuscript_methods import group_statistics
from manuscript_methods.maf import maf_discrepancies

data = dataset_maf.df.select(maf_discrepancies(f.col("majorLdPopulationMAF.value")))

group_statistics(data, f.col("mafDiscrepancy")).show(truncate=False)


+-------------------------------------+-------+-----+
|mafDiscrepancy                       |count  |%    |
+-------------------------------------+-------+-----+
|NULL                                 |2821580|99.57|
|MAJOR_ANCESTRY_MISSING_FROM_GNOMAD_AF|9434   |0.33 |
|VARIANT_MISSING_FROM_GNOMAD_AF       |2744   |0.10 |
+-------------------------------------+-------+-----+

